In [1]:
import pandas as pd 
import numpy as np 

In [3]:
# загрузили датасет u.data в переменную ratings 

column_names = ['userID', 'movieID', 'rating', 'timestamp'] 
ratings = pd.read_csv('u.data.csv', header = None, names = column_names, sep='\t') 
ratings.head()


,userID,movieID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
ratings.dtypes

userID       int64
movieID      int64
rating       int64
timestamp    int64
dtype: object

In [5]:
# посчитать суммарную оценку для каждого фильма, рез-т сохранить в таблицу 

In [5]:
total_eval = ratings.groupby('movieID').sum()[['rating']].reset_index().rename(columns={'rating': 'total_sum'})
total_eval.head(5)

,movieID,total_sum
0,1,1753
1,2,420
2,3,273
3,4,742
4,5,284


In [ ]:
# посчитать общее количество оценок для каждого фильма, результат сохраните в отдельную таблицу

In [6]:
total_amount = ratings.groupby('movieID').count()[['rating']].reset_index().rename(columns={'rating': 'total_amount'})
total_amount.head(5)

,movieID,total_amount
0,1,452
1,2,131
2,3,90
3,4,209
4,5,86


In [36]:
# в таблице ratings найти самого активного юзера (который выставил больше всего оценок)

In [7]:
ratings['userID'].value_counts().head(1) # самый активный юзер - 405

405    737
Name: userID, dtype: int64

In [ ]:
# ratings объединяем с таблицами total_eval и total_amount, используем merge по movie_id

In [8]:
top_user = ratings.query('userID == 405') # оставляем только самого активного пользователя
merged = top_user.merge(total_eval, how = 'left', on = 'movieID').merge(total_amount, how = 'left', on = 'movieID')
# merged1 = merged.merge(total_amount, how = 'inner', on = 'itemID')#
merged.head(7)


,userID,movieID,rating,timestamp,total_sum,total_amount
0,405,56,4,885544911,1600,394
1,405,592,1,885548670,30,9
2,405,1582,1,885548670,1,1
3,405,171,1,885549544,252,65
4,405,580,1,885547447,108,32
5,405,1409,1,885549045,20,7
6,405,953,3,885546487,73,22


In [9]:
# загрузили датасет u.item в переменную movies 

namesList = ['movieID', 'movie_title', 'release_date', 'video_release_date', 'URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                'Drama', 'Fantasy', 'Film_Noir','Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item.csv', sep='|', header=None, names = namesList, encoding='ISO-8859-9')
movies.head()


,movieID,movie_title,release_date,video_release_date,URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
import datetime

movies['release_date'] = pd.to_datetime(movies['release_date'])
movies['year'] = movies['release_date'].dt.year # создаем новый столбец year с годом 
movies['year'] = movies.year.astype('Int64') # перевести для порядка годы к integer
movies.head(3)


,movieID,movie_title,release_date,video_release_date,URL,unknown,Action,Adventure,Animation,Children,...,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,year
0,1,Toy Story (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995


In [11]:
# объединяем две таблицы в одну финальную и оставояем нужные для построения модели столбцы 

df_final = merged.merge(movies, how = 'left', on = 'movieID')
df_final[['movieID', 'rating', 'total_sum', 'total_amount','unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
         'Drama', 'Fantasy', 'Film_Noir','Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western', 'year']].head()


,movieID,rating,total_sum,total_amount,unknown,Action,Adventure,Animation,Children,Comedy,...,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,year
0,56,4,1600,394,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
1,592,1,30,9,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1995
2,1582,1,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1947
3,171,1,252,65,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1991
4,580,1,108,32,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1995


Предсказываем значение столбца rating на основе других признаков методом линейной регрессии

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [14]:
X = df_final[['total_sum', 'total_amount','unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
         'Drama', 'Fantasy', 'Film_Noir','Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western', 'year']]
# признаки, на основании которых будем предсказывать 

y = df_final['rating'] # будем предсказывать оценку пользователя (зависимая переменная)

In [15]:
# разделяем на данные для обучения и тестирования модели 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [16]:
model = LinearRegression()
model.fit(X_train, y_train)

print(model.coef_)
print(model.intercept_) # метод обучается на данных и подбирает оптимальные коэффициенты

[-1.86137964e-03  1.75250152e-02 -1.38430933e-15 -2.26875248e-01
 -3.14486190e-01 -2.10606156e-01  4.37441570e-01  2.03471345e-01
 -1.93514996e-02  5.13864540e-01  1.52834298e-01 -2.93304664e-01
 -4.27173048e-01 -2.48430312e-02  2.67104877e-01  3.07876293e-01
 -1.25712843e-01  8.89485262e-02  2.10797836e-01 -3.67069369e-01
 -1.13169782e-01 -6.43566989e-03]
13.760996060252738


In [17]:
y_pred = model.predict(X_test)
y_pred

array([1.13530377, 1.77359699, 0.99445802, 1.30465257, 3.02900949,
       3.05685411, 1.18319379, 2.02960596, 4.40648701, 1.83064881,
       3.3753275 , 2.03742777, 1.4134322 , 2.73058821, 0.94253646,
       1.35796392, 1.58119801, 2.57673852, 1.95564194, 1.34572637,
       1.79078724, 1.97281828, 1.1760723 , 1.68548922, 1.38002402,
       1.08147575, 1.51220237, 1.66772646, 1.48532406, 2.61621718,
       1.15749325, 3.98735316, 1.8171042 , 3.24892239, 1.21905054,
       1.18779989, 1.30178514, 3.36926453, 0.48048255, 1.96442771,
       3.11638896, 0.48830992, 1.38351837, 1.98908942, 3.55575387,
       2.2723234 , 1.24013908, 3.73835572, 1.17008656, 4.27741873,
       3.66021427, 1.48733404, 1.32643859, 5.57979975, 1.34077478,
       1.51143256, 2.41797348, 3.38413706, 1.16038967, 1.21209719,
       1.21633763, 1.22083299, 1.4605547 , 2.51400861, 1.3343876 ,
       1.71759091, 1.46645502, 1.97572177, 1.12465339, 1.14181883,
       1.36145165, 0.90553044, 1.93490968, 1.75096823, 2.21873

In [18]:
r2 = round(model.score(X_test, y_test),2) # нахождение коэффициента детерминации
print(r2)

0.16


In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
y_true = y_test

In [21]:
# Расчет среднего квадрата ошибки (MSE)
mean_squared_error(y_true, y_pred)

1.4544452438444317

In [7]:
# объединим два фрейма movies and ratings в один и сохраним из ноутбука в формате csv

In [12]:
movies_ratings = pd.merge(ratings, movies, how = 'inner', on = 'movieID')
movies_ratings.sample(10)

,userID,movieID,rating,timestamp,movie_title,release_date,video_release_date,URL,unknown,Action,...,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,year
53617,679,184,4,884487491,Army of Darkness (1993),1993-01-01,NaN,http://us.imdb.com/M/title-exact?Army%20of%20D...,0,1,...,0,1,0,0,0,1,0,0,0,1993
46496,942,97,5,891283239,Dances with Wolves (1990),1990-01-01,NaN,http://us.imdb.com/M/title-exact?Dances%20with...,0,0,...,0,0,0,0,0,0,0,0,1,1990
95040,118,551,5,875385306,Lord of Illusions (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Lord%20of%20I...,0,0,...,0,1,0,0,0,0,0,0,0,1995
65294,572,9,5,879449610,Dead Man Walking (1995),1995-01-01,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,...,0,0,0,0,0,0,0,0,0,1995
71354,864,380,3,888889744,Star Trek: Generations (1994),1994-01-01,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,0,1,...,0,0,0,0,0,1,0,0,0,1994
64078,63,269,3,875746948,"Full Monty, The (1997)",1997-01-01,NaN,http://us.imdb.com/M/title-exact?Full+Monty%2C...,0,0,...,0,0,0,0,0,0,0,0,0,1997
45082,380,71,4,885479082,"Lion King, The (1994)",1994-01-01,NaN,"http://us.imdb.com/M/title-exact?Lion%20King,%...",0,0,...,0,0,1,0,0,0,0,0,0,1994
48746,770,596,4,875972988,"Hunchback of Notre Dame, The (1996)",1996-06-21,NaN,http://us.imdb.com/M/title-exact?Hunchback%20o...,0,0,...,0,0,1,0,0,0,0,0,0,1996
20304,763,658,3,878915600,Pump Up the Volume (1990),1990-01-01,NaN,http://us.imdb.com/M/title-exact?Pump%20Up%20t...,0,0,...,0,0,0,0,0,0,0,0,0,1990
70727,805,213,3,881696699,"Room with a View, A (1986)",1986-01-01,NaN,http://us.imdb.com/M/title-exact?Room%20with%2...,0,0,...,0,0,0,0,1,0,0,0,0,1986


In [21]:
movies_ratings.to_csv('movies_ratings.csv', index=False)